## SQL и Pandas

В этом документе представлено сравнение базовых операций библиотеки Pandas со схожими запросами в языке структурированных запросов *SQL*.

Для начала импортируем библиотеку:

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'0.24.2'

Импортируем данные из файла *tips.csv* в привычную pandas-структуру *DataFrame*. Будем считать, что у нас есть аналогичная таблица базы данных с тем же именем и структурой для SQL.

In [2]:
tips = pd.read_csv('tips.csv')

Отобразим первые и последние 8 элементов таблицы:

In [3]:
tips.head(8)    # First 8 elements

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
6,8.77,2.00,Male,No,Sun,Dinner,2
7,26.88,3.12,Male,No,Sun,Dinner,4


In [4]:
tips.tail(8)    # Last 8 elements

,total_bill,tip,sex,smoker,day,time,size
236,12.60,1.00,Male,Yes,Sat,Dinner,2
237,32.83,1.17,Male,Yes,Sat,Dinner,2
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2


### SELECT

SELECT - оператор запроса в языке SQL, возвращающий набор данных из базы. В SQL выбор осуществляется с помощью списка столбцов через запятую (или \*, если требуется выбрать все столбцы):

```SQL
SELECT total_bill, tip, smoker, size
FROM tips
LIMIT 8;
```

В pandas выбор столбцов производится путем передачи списка имен столбцов в *DataFrame*:

In [5]:
tips[['total_bill', 'tip', 'smoker', 'size']].head(8)    # Select columns from Dataframe

,total_bill,tip,smoker,size
0,16.99,1.01,No,2
1,10.34,1.66,No,3
2,21.01,3.50,No,3
3,23.68,3.31,No,2
4,24.59,3.61,No,4
5,25.29,4.71,No,4
6,8.77,2.00,No,2
7,26.88,3.12,No,4


Вызов DataFrame без списка имен столбцов отобразит всю таблицу (аналогично в SQL - \*).

### WHERE

Фильтрация по критериям в SQL производится с помощью конструкции WHERE. Критерии должны быть описаны в форме предикатов (утверждений).

```SQL
SELECT *
FROM tips
WHERE size = 2
LIMIT 5;
```

Фильтрация в DataFrame производится несколькими способами. Наиболее интуитивный использует логическое индексирование. 

В качестве примера, отобразим все элементы таблицы, удовлетворяющие условию `size == 2`.

In [6]:
tips[tips['size'] == 2].head(4)    # Filter data by size == 2

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
3,23.68,3.31,Male,No,Sun,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
8,15.04,1.96,Male,No,Sun,Dinner,2


Приведенный выше оператор передает в *DataFrame* набор значений `True` и `False`, представленный в виде фильтра pandas *Series*. 

Первые 4 элемента булевого фильтра `size == 2`:

In [7]:
(tips['size'] == 2).head(4)    # Show first 4 elements of boolean filter

0     True
1    False
2    False
3     True
Name: size, dtype: bool

Еще один пример по признаку `sex == 'Male'`:

In [8]:
tips[tips['sex'] == 'Male'].head(4)    # Filter data by sex == Male

,total_bill,tip,sex,smoker,day,time,size
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
5,25.29,4.71,Male,No,Sun,Dinner,4


Pandas позволяет подсчитать количество истиных и ложных значений в таблице, удовлетворяющих выбранному условию:

In [9]:
is_size = tips['size'] == 2
is_size.value_counts()

True     156
False     88
Name: size, dtype: int64

Тоже самое по другому признаку:

In [10]:
is_male = tips['sex'] == 'Male'
is_male.value_counts()

True     157
False     87
Name: sex, dtype: int64

Отдельный доступ к истинным или ложным значениям (несмотря на то, что значение True отображается до False, индексация производится наоборот: по нулевому индексу - False, по первому - True):

In [11]:
is_male.value_counts()[0]    # Number of False values

87

In [12]:
is_male.value_counts()[1]    # Number of True values

157

В pandas *DataFrame* может быть передано несколько логических условий с помощью | (ИЛИ) и & (И). В SQL для этой цели используются операторы OR и AND.

```SQL
-- tips of more than 5.00 dollars at Dinner meals
SELECT *
FROM tips
WHERE time = 'Dinner' AND tip > 5.00;
```

In [13]:
# tips of more than $5.00 at Dinner meals
tips[(tips['time'] == 'Dinner') & (tips['tip'] > 5.00)]

,total_bill,tip,sex,smoker,day,time,size
23,39.42,7.58,Male,No,Sat,Dinner,4
44,30.40,5.60,Male,No,Sun,Dinner,4
47,32.40,6.00,Male,No,Sun,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
116,29.93,5.07,Male,No,Sun,Dinner,4
155,29.85,5.14,Female,No,Sun,Dinner,5
170,50.81,10.00,Male,Yes,Sat,Dinner,3
172,7.25,5.15,Male,Yes,Sun,Dinner,2
181,23.33,5.65,Male,Yes,Sun,Dinner,2


Другой пример:

```SQL
-- tips by parties of at least 5 diners OR bill total was more than $45
SELECT *
FROM tips
WHERE size >= 5 OR total_bill > 45;
```

In [14]:
# tips by parties of at least 5 diners OR bill total was more than $45
tips[(tips['size'] >= 5) | (tips['total_bill'] > 45)]

,total_bill,tip,sex,smoker,day,time,size
59,48.27,6.73,Male,No,Sat,Dinner,4
125,29.80,4.20,Female,No,Thur,Lunch,6
141,34.30,6.70,Male,No,Thur,Lunch,6
142,41.19,5.00,Male,No,Thur,Lunch,5
143,27.05,5.00,Female,No,Thur,Lunch,6
155,29.85,5.14,Female,No,Sun,Dinner,5
156,48.17,5.00,Male,No,Sun,Dinner,6
170,50.81,10.00,Male,Yes,Sat,Dinner,3
182,45.35,3.50,Male,Yes,Sun,Dinner,3
185,20.69,5.00,Male,No,Sun,Dinner,5


Проверка на *NULL* производится с помощью методов `notna()` и `isna()`. Создадим массив с NaN-значениями:

In [15]:
fr = pd.DataFrame({
        'col1': ['A', 'B', np.NaN, 'C', 'D'],
        'col2': ['F', np.NaN, 'G', 'H', 'I']
})

fr

,col1,col2
0,A,F
1,B,NaN
2,NaN,G
3,C,H
4,D,I


Предположим, у нас есть таблица с такой же структурой. С помощью следующего SQL-запроса можно отобразить записи, в которых отражены значения *NULL* во второй колонке *col 2*:

```SQL
SELECT *
FROM frame
WHERE col2 IS NULL;
```

In [16]:
fr[fr['col2'].isna()]   # Show NaN from 'col 2'

,col1,col2
1,B,NaN


Получить не-NULL значения из *col 1* можно с помощью метода `notna()`. В SQL это команда:

```SQL
SELECT *
FROM frame
WHERE col1 IS NOT NULL;
```

In [17]:
fr[fr['col1'].notna()]   # Show not-NaN from 'col 1'

,col1,col2
0,A,F
1,B,NaN
3,C,H
4,D,I
